# DickensAssignmentValidator

MUCEP Task 1 (Dr. Pierre-Paul Bitton)

Author: Shawon Ibn Kamal\
Email: sikamal@mun.ca

### Updates to exisiting program

I made a few changes in the existing curator program to work with it efficiently and sorted out a few bugs. Here's the list:

- The files getting read from is renamed to "DataFiles" from "Files".
- The output csv files are being stored in a folder named "OutputFiles".
- Renamed "NotMatched.csv" to "MissingMeta.csv" in order to avoid confusion with "MissingFiles.csv".
- Stored the program in git, currently a private repo to me. I think it is a good way to track updates,\
  we can work on it if you are interested
- Fixed few minor bugs in DickensAssignment.py program


In [1]:
import pandas as pd
import numpy as np

### Run DickensAssignment.py



In [2]:
exec(open('DickensAssignment.py').read())

4865 no. of files
4650 match found
215 match not found
Complete


### Compare OutputFiles with OutputFiles_2020_07_14

In [3]:
# Load old outputs
df_old_result = pd.read_csv('OutputFiles_2020_07_14/Result.csv', engine='python')
df_old_missing_files = pd.read_csv('OutputFiles_2020_07_14/MissingFiles.csv', engine='python')
df_old_missing_meta = pd.read_csv('OutputFiles_2020_07_14/MissingMeta.csv', engine='python')

# Load new outputs
df_new_result = pd.read_csv('OutputFiles/Result.csv', engine='python')
df_new_missing_files = pd.read_csv('OutputFiles/MissingFiles.csv', engine='python')
df_new_missing_meta = pd.read_csv('OutputFiles/MissingMeta.csv', engine='python')

# Load filenames
filenames = [name for path, subdirs, files in os.walk("DataFiles")
             for name in files]

df_data_files = pd.DataFrame({'filename':filenames}).sort_values(by='filename')

# Load template
df_template = pd.read_csv('template.csv', engine='python')

# Sort Result
df_old_result = df_old_result.sort_values(by='FileName')
df_new_result = df_new_result.sort_values(by='FileName')

In [4]:
df_diff_result = pd.concat([df_old_result.dropna(axis=0),df_new_result.dropna(axis=0)], sort=True).drop_duplicates(keep=False)
df_diff_missing_files = pd.concat([df_old_missing_files,df_new_missing_files], sort=True).drop_duplicates(keep=False)
df_diff_missing_meta = pd.concat([df_old_missing_meta,df_new_missing_meta], sort=True).drop_duplicates(keep=False)

if (df_diff_result.shape[0] == 0):
    print("Results are the same")
else:
    print("Results have ", df_diff_result.shape[0], " differences")
    
if (df_diff_missing_files.size == 0):
    print("MissingFiles are the same")
else:
    print("MissingFiles have ", df_diff_missing_files.shape[0], " differences")

if (df_new_missing_meta.shape[0] == 0):
    print("NotMatchedFiles are the same")
else:
    print("NotMatchedFiles have ", df_diff_missing_meta.shape[0], " differences")



Results are the same
MissingFiles have  21  differences
NotMatchedFiles have  617  differences


### Check to see if MissingMetaData entries are due to typo

In [5]:
def includes(fullstring, substrings=[]):
    count = 0
    for each_substring in substrings:
        if fullstring.find(each_substring) != -1:
            count += 1
    return count

# Testing
print(includes("I like data", ["like", "data"]))

2


In [6]:
df_template['key'] = 0
df_new_missing_meta['key'] = 0

# Cartessian product of two dataframes
df_merged_template_and_missing_meta = df_template.merge(df_new_missing_meta, how='outer')

In [7]:
df_merged_template_and_missing_meta['similarity'] = df_merged_template_and_missing_meta.apply(lambda row : includes(row['notmatched'], [str(row['catalogueNumber']), row['institutionCode']]), axis=1)

In [8]:
df_merged_template_and_missing_meta = df_merged_template_and_missing_meta[['institutionCode', 'catalogueNumber', 'notmatched', 'similarity']][df_merged_template_and_missing_meta['similarity'] > 1].sort_values(by='notmatched', ascending=False)
print(df_merged_template_and_missing_meta['notmatched'].count())

# Export data
df_merged_template_and_missing_meta.to_csv('ValidatorExports/MissingMetaSimilar.csv', index=False)

# Print first 50 data
df_merged_template_and_missing_meta.head(10)

25


,institutionCode,catalogueNumber,notmatched,similarity
6059,CM,72696,AM.U.CM972696.00000005.Master.Transmission,2
6058,CM,72696,AM.U.CM972696.00000004.Master.Transmission,2
6057,CM,72696,AM.U.CM972696.00000003.Master.Transmission,2
6056,CM,72696,AM.U.CM972696.00000002.Master.Transmission,2
6055,CM,72696,AM.U.CM972696.00000001.Master.Transmission,2
6054,CM,72696,AM.T.CM972696.00000005.Master.Transmission,2
6053,CM,72696,AM.T.CM972696.00000004.Master.Transmission,2
6052,CM,72696,AM.T.CM972696.00000003.Master.Transmission,2
6051,CM,72696,AM.T.CM972696.00000002.Master.Transmission,2
6050,CM,72696,AM.T.CM972696.00000001.Master.Transmission,2


In [9]:
# Find no similarities at all
df_merged_template_and_missing_meta = df_merged_template_and_missing_meta.drop_duplicates('notmatched')
df_missing_meta_nonsimilar = pd.concat([df_new_missing_meta['notmatched'],df_merged_template_and_missing_meta['notmatched']]).drop_duplicates(keep=False)

# Export non-similar data
df_missing_meta_nonsimilar.to_csv('ValidatorExports/MissingMetaNonSimilar.csv', index=False, header=True)

# Print first 50 data
df_missing_meta_nonsimilar.head(10)

0     AM.H.MZUSP92643.00000001.csv
1     AM.H.MZUSP92643.00000002.csv
2     AM.H.MZUSP92643.00000003.csv
3     AM.H.MZUSP92643.00000004.csv
4     AM.H.MZUSP92643.00000005.csv
10    AM.M.MZUSP92643.00000001.csv
11    AM.M.MZUSP92643.00000002.csv
12    AM.M.MZUSP92643.00000003.csv
13    AM.M.MZUSP92643.00000004.csv
14    AM.M.MZUSP92643.00000005.csv
Name: notmatched, dtype: object

### Find similar files for MissingFiles

In [10]:
df_data_files['key'] = 0
df_new_missing_files['key'] = 0

# Cartessian product of two dataframes
df_merged_data_files_and_missing_files = df_data_files.merge(df_new_missing_files, how='outer')
df_merged_data_files_and_missing_files.head()

,filename,key,FileName,institutionCode,collectionCode,catalogueNumber,class,order,family,genus,...,verbatimElevation,eventDate,measurementDeterminedDate,Patch,LightAngle1,LightAngle2,ProbeAngle1,ProbeAngle2,Replicate,Comments
0,AM.H.AMNH278606.00000001.Master.Transmission,0,NaN,MZUSP,NaN,97287,Aves,Trogoniformes,Trogonidae,Trogon,...,NaN,2013-6-26,NaN,NaN,0,0,0,0,NaN,NaN
1,AM.H.AMNH278606.00000001.Master.Transmission,0,NaN,MZUSP,NaN,76792,Aves,Trogoniformes,Trogonidae,Trogon,...,NaN,2007-1-20,NaN,NaN,0,0,0,0,NaN,NaN
2,AM.H.AMNH278606.00000001.Master.Transmission,0,NaN,MZUSP,NaN,86474,Aves,Trogoniformes,Trogonidae,Trogon,...,NaN,2009-7-16,NaN,NaN,0,0,0,0,NaN,NaN
3,AM.H.AMNH278606.00000001.Master.Transmission,0,NaN,MCZ,NaN,173836,Aves,Trogoniformes,Trogonidae,Trogon,...,NaN,1932-3-4,NaN,NaN,0,0,0,0,NaN,NaN
4,AM.H.AMNH278606.00000001.Master.Transmission,0,NaN,MZUSP,NaN,15953,Aves,Trogoniformes,Trogonidae,Trogon,...,NaN,1939-3-19,NaN,NaN,0,0,0,0,NaN,NaN


In [11]:
# Calculate similarity
df_merged_data_files_and_missing_files['similarity'] = df_merged_data_files_and_missing_files.apply(lambda row : includes(row['filename'], [str(row['catalogueNumber'])]), axis=1)

In [12]:
# Sort
df_merged_data_files_and_missing_files = df_merged_data_files_and_missing_files[['institutionCode', 'catalogueNumber', 'filename', 'similarity']][df_merged_data_files_and_missing_files['similarity'] > 0].sort_values(by='institutionCode', ascending=False).reset_index()
print(df_new_missing_files.shape[0])
print("Length of similarities", df_merged_data_files_and_missing_files.shape[0])

# Export data
df_merged_data_files_and_missing_files.to_csv('ValidatorExports/MissingFilesSimilarity.csv', index=False)

# Print first 50 data
df_merged_data_files_and_missing_files.head(10)

43
Length of similarities 0


,index,institutionCode,catalogueNumber,filename,similarity


In [13]:
# Find no similarities at all
df_merged_data_files_and_missing_files_unique = df_merged_data_files_and_missing_files.drop_duplicates(subset=['institutionCode', 'catalogueNumber'])
df_missing_files_nonsimilar = pd.concat([df_new_missing_files[['institutionCode', 'catalogueNumber']],df_merged_data_files_and_missing_files_unique[['institutionCode', 'catalogueNumber']]]).drop_duplicates(keep=False)

# Export non-similar data
df_missing_files_nonsimilar.to_csv('ValidatorExports/MissingFilesNonSimilar.csv', index=False, header=True)

print("There are ", df_missing_files_nonsimilar.shape[0], " meta data with no similarities.")

# Print first 50 data
df_missing_files_nonsimilar.head(10)

There are  43  meta data with no similarities.


,institutionCode,catalogueNumber
0,MZUSP,97287
1,MZUSP,76792
2,MZUSP,86474
3,MCZ,173836
4,MZUSP,15953
5,MZUSP,44168
6,MZUSP,44172
7,MZUSP,44175
8,MCZ,173842
9,MCZ,173839
